<a href="https://colab.research.google.com/github/PamelaVQ/Base-ML/blob/master/Pytorch_Basics/RNN_for_Sentence_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use Recurrent Neural Network for Sentence Generation

Reference Links:

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

https://gist.github.com/karpathy/d4dee566867f8291f086

https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py


In [1]:
from tensorflow.keras.utils import get_file
import io
import sklearn
import sklearn.feature_extraction
import numpy as np
from torch import nn
import torchvision

In [2]:
def pattern_text(start_pattern, end_pattern, data):
  # pattern = f'{start_pattern}(.*){end_pattern}'
  # result = re.search(re.escape(pattern), data)
  result = data[data.find(start_pattern)+len(start_pattern):data.rfind(end_pattern)]
  return result

In [3]:
path = get_file("agatha_christie", origin="https://www.gutenberg.org/files/863/863-0.txt")
with io.open(path, encoding='utf-8') as read_file:
  data_agatha_christie = read_file.readlines()
# start_text = """*** START OF THIS PROJECT GUTENBERG EBOOK THE MYSTERIOUS AFFAIR AT STYLES ***""".lower()
# end_text = """*** END OF THIS PROJECT GUTENBERG EBOOK THE MYSTERIOUS AFFAIR AT STYLES ***""".lower()
# data_agatha_christie = pattern_text(start_text, end_text, data)
# print(f'agatha_christie corpus length:{len(data_agatha_christie)}')

path = get_file("lewis_carroll", origin="https://www.gutenberg.org/files/11/11-0.txt")
with io.open(path, encoding='utf-8') as read_file:
  data_lewis_carroll = read_file.readlines()
# start_text = """*** START OF THIS PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***""".lower()
# end_text = """*** END OF THIS PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***""".lower()
# data_lewis_carroll = pattern_text(start_text, end_text, data)
# print(f'lewis_carroll corpus length:{len(data_lewis_carroll)}')

In [4]:
def reverse_dictionary(my_map):
  return {v: k for k, v in my_map.items()}

def create_vocab(corpus):
  # corpus = [x for x in data if len(x)>0]
  vectorizer = sklearn.feature_extraction.text.CountVectorizer(min_df=1)
  X = vectorizer.fit_transform(corpus)
  analyzer = vectorizer.build_analyzer()
  output_corpus = [x for x in [list(map(lambda x: vectorizer.vocabulary_.get(x), analyzer(line))) for line in corpus] if x]
  word2idx = vectorizer.vocabulary_
  print(f'word2idx: {word2idx}')
  idx2word = reverse_dictionary(word2idx)
  print(f'idx2word: {idx2word}')
  return word2idx, idx2word, output_corpus

word2idx, idx2word, output_corpus = create_vocab(data_agatha_christie)

word2idx: {'the': 4985, 'project': 3843, 'gutenberg': 2354, 'ebook': 1661, 'of': 3413, 'mysterious': 3270, 'affair': 215, 'at': 454, 'styles': 4791, 'by': 777, 'agatha': 232, 'christie': 917, 'this': 5010, 'is': 2776, 'for': 2098, 'use': 5284, 'anyone': 341, 'anywhere': 344, 'in': 2621, 'united': 5223, 'states': 4695, 'and': 315, 'most': 3228, 'other': 3468, 'parts': 3548, 'world': 5552, 'no': 3340, 'cost': 1169, 'with': 5521, 'almost': 275, 'restrictions': 4159, 'whatsoever': 5454, 'you': 5593, 'may': 3108, 'copy': 1155, 'it': 2779, 'give': 2251, 'away': 505, 'or': 3454, 're': 3969, 'under': 5194, 'terms': 4963, 'license': 2940, 'included': 2633, 'online': 3439, 'www': 5577, 'org': 3459, 'if': 2575, 'are': 386, 'not': 3357, 'located': 2991, 'll': 2987, 'have': 2417, 'to': 5055, 'check': 882, 'laws': 2893, 'country': 1186, 'where': 5457, 'before': 575, 'using': 5292, 'title': 5053, 'author': 484, 'release': 4065, 'date': 1282, 'july': 2816, '27': 17, '2008': 14, '863': 36, 'last': 2881

In [8]:
def get_vocab_details(corpus):
  max_length = max([len(corpus) for corpus in corpus])
  vocab_size = len(corpus)
  return max_length, vocab_size

max_length, vocab_size = get_vocab_details(output_corpus)

In [9]:
class RNN(nn.Module):
    def __init__(self, max_length, vocab_size):
      super(RNN, self).__init__()
      # self.embed = nn.Embedding(input_size, 128)
      self.lstm = nn.LSTM(128, input_shape=(max_length, vocab_size))
      self.dense = nn.Dense(vocab_size, activation='softmax')

    def forward(self, x):
      x = self.lstm(x)
      x = self.dense(x)

In [10]:
from autograd import Variable
input_test = Variable(torch.ones(max_length, vocab_size))
rnn = RNN(input_test)
print(rnn)

ImportError: ignored